In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style()
plt.rcParams['figure.figsize'] = (12, 6)

In [3]:
vars_procedimientos = {f"PROCEDIMIENTO{i}": str for i in range(1, 31)}

In [4]:
df_procesada = pd.read_csv("../data/processed/df_procesada.csv", sep=";", dtype=vars_procedimientos)

C:\Users\ppizarro\AppData\Local\Temp\ipykernel_7140\2020992259.py:1: DtypeWarning: Columns (24,25,26,27,28,29,30,31,32,33,37,41,43,47,51,75,76,77,78,79,80,81,82,83,84,85,86,87,123) have mixed types. Specify dtype option on import or set low_memory=False.
  df_procesada = pd.read_csv("../data/processed/df_procesada.csv", sep=";", dtype=vars_procedimientos)


In [41]:
COLUMNAS_SERVICIO_TRASLADO = {
    "SERVICIOINGRESO": 0,
    "SERVICIOTRASLADO1": 1,
    "SERVICIOTRASLADO2": 2,
    "SERVICIOTRASLADO3": 3,
    "SERVICIOTRASLADO4": 4,
    "SERVICIOTRASLADO5": 5,
    "SERVICIOTRASLADO6": 6,
    "SERVICIOTRASLADO7": 7,
    "SERVICIOTRASLADO8": 8,
    "SERVICIOTRASLADO9": 9,
    "SERVICIOALTA": 10,
}

COLUMNAS_FECHA_TRASLADO = {
    "FECHA_INGRESO": 0,
    "FECHATRASLADO1": 1,
    "FECHATRASLADO2": 2,
    "FECHATRASLADO3": 3,
    "FECHATRASLADO4": 4,
    "FECHATRASLADO5": 5,
    "FECHATRASLADO6": 6,
    "FECHATRASLADO7": 7,
    "FECHATRASLADO8": 8,
    "FECHATRASLADO9": 9,
    "FECHAALTA": 10,
}


# Filtra la base nacional para dejar el Torax
df_torax = df_procesada.query("COD_HOSPITAL == 112103")
# Permite identificar cada egreso
df_torax = df_torax.reset_index().reset_index(names="id_egreso")

In [48]:
# Convierte los servicios y las fechas en formato largo
servicios_egreso_long = df_torax.melt(
    id_vars="id_egreso",
    value_vars=COLUMNAS_SERVICIO_TRASLADO,
    var_name="tipo_servicio",
    value_name="servicio",
)

fechas_egreso_long = df_torax.melt(
    id_vars="id_egreso",
    value_vars=COLUMNAS_FECHA_TRASLADO,
    var_name="tipo_fecha",
    value_name="fecha",
)

In [67]:
# Une los servicios y fechas respectivas para que esten en formato contiguo
servicio_y_fechas_egreso_contiguas = pd.merge(
    left=fechas_egreso_long,
    right=servicios_egreso_long,
    how="inner",
    left_index=True,
    right_index=True,
)

# Indica el tipo de ordenamiento del tipo de servicio ("SERVICIONGR", "SERVICIO1", "SERVICIOALTA")
servicio_y_fechas_egreso_contiguas["tipo_servicio"] = pd.Categorical(
    servicio_y_fechas_egreso_contiguas["tipo_servicio"],
    categories=COLUMNAS_SERVICIO_TRASLADO.keys(),
    ordered=True,
)

# Ordena cada egreso segun su tipo de servicio ("SERVICIONGR", "SERVICIO1", "SERVICIOALTA")
servicio_y_fechas_egreso_contiguas = servicio_y_fechas_egreso_contiguas.sort_values(
    ["id_egreso_x", "tipo_servicio"]
)

In [68]:
servicio_y_fechas_egreso_contiguas

,id_egreso_x,tipo_fecha,fecha,id_egreso_y,tipo_servicio,servicio
0,0,FECHA_INGRESO,2019-01-14,0,SERVICIOINGRESO,CIRUGÍA CARDIOVASCULAR
13466,0,FECHATRASLADO1,NaN,0,SERVICIOTRASLADO1,NaN
26932,0,FECHATRASLADO2,NaN,0,SERVICIOTRASLADO2,NaN
40398,0,FECHATRASLADO3,NaN,0,SERVICIOTRASLADO3,NaN
53864,0,FECHATRASLADO4,NaN,0,SERVICIOTRASLADO4,NaN
...,...,...,...,...,...,...
94261,13465,FECHATRASLADO6,NaN,13465,SERVICIOTRASLADO6,NaN
107727,13465,FECHATRASLADO7,NaN,13465,SERVICIOTRASLADO7,NaN
121193,13465,FECHATRASLADO8,NaN,13465,SERVICIOTRASLADO8,NaN
134659,13465,FECHATRASLADO9,NaN,13465,SERVICIOTRASLADO9,NaN
